# ML to predict EMISIONES_EURO
The objective of this notebook is no create a ML model capable pf predicting EMISIONES_EURO in order to fill the _null_ values in the dataset

In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
import polars as pl
import numpy as np
import random
import os

Load and divide the dataset

In [ ]:
import polars as pl
import os
from utils.dictionaries import types_parque_post

path = os.path.join("..","Data", "DGT")
clean_park = os.path.join(path,'Parque_exacto','clean_park.csv')

parke = pl.read_csv(clean_park,separator='|', schema=types_parque_post)
parke

In [2]:
path = os.path.join("..","Data", "DGT")
complete_park = os.path.join(path,'Parque_exacto','clean_park.csv')

In [ ]:
columns = ['FABRICANTE','MARCA','MODELO','TIPO','VARIANTE','VERSION','FECHA_MATR','FECHA_PRIM_MATR',
           'CLASE_MATR','SUBTIPO_DGT','TIPO_DGT','CAT_EURO','TARA','PESO_MAX','MOM','MMTA',
           'CILINDRADA','PROPULSION','TIPO_DISTINTIVO','EMISIONES_CO2','EMISIONES_EURO']

data = pl.read_csv(complete_park, separator='|',null_values=["",''])
data = data.select(columns)
data = data.filter(pl.col('EMISIONES_EURO').is_not_null()).head(1000000)
data

ComputeError: could not parse `"C"` as dtype `i64` at column 'KW' (column number 31)

The current offset in the file is 163109 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `"C"` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [6]:
def split_dataframe(data,proportion = 0.33,seed = 42, target_column='EMISIONES_EURO'):
    data = data.sample(fraction=1, shuffle=True, seed=seed)
    lenght = data.height
    test_size= round(lenght*proportion)
    test, train = data.head(test_size), data.tail(-test_size)

    X_train = train.drop(target_column)
    y_train = train.select(target_column)
    X_test = test.drop(target_column)
    y_test = test.select(target_column)
    return X_train,y_train,X_test,y_test

In [ ]:
X_train,y_train,X_test,y_test = split_dataframe(data)

In [3]:
x_train_path = os.path.join(path,'Parque_exacto','x_train.csv')
x_test_path = os.path.join(path,'Parque_exacto','x_test.csv')
y_train_path = os.path.join(path,'Parque_exacto','y_train.csv')
y_test_path = os.path.join(path,'Parque_exacto','y_test.csv')

In [ ]:
X_train.write_csv(x_train_path,separator='|')
X_test.write_csv(x_test_path,separator='|')
y_train.write_csv(y_train_path,separator='|')
y_test.write_csv(y_test_path,separator='|')

In [20]:
X_train = pl.read_csv(x_train_path, separator='|',null_values='null')
y_train = pl.read_csv(y_train_path, separator='|',null_values='null')
X_test = pl.read_csv(x_test_path, separator='|',null_values='null')
y_test = pl.read_csv(y_test_path, separator='|',null_values='null')

Start the model

In [11]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

ValueError: could not convert string to float: ''